In [10]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img,img_to_array,ImageDataGenerator
from tensorflow.keras.layers import Dense,Flatten,Dropout,Input,AveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt    
import os
import warnings
warnings.filterwarnings('ignore')

In [11]:
learning_rate = 0.0001
epochs = 20
batchs = 32

In [12]:
data = []
labels = []

path = r"dataset"
for category in os.listdir(path) :
    folder_path = os.path.join(path,category)
    for img in os.listdir(folder_path) :
        img_path = os.path.join(folder_path,img)
        img = load_img(img_path,target_size = (224,224))
        img_arr = img_to_array(img)
        img_preprocess = preprocess_input(img_arr)
        data.append(img_preprocess)
        labels.append(category)

In [13]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [14]:
data = np.array(data, dtype="float32")
labels = np.array(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels,
                                    test_size=0.20, stratify=labels, random_state=42)

In [15]:
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

In [16]:
baseModel = MobileNetV2(weights="imagenet", include_top=False,
                        input_tensor=Input(shape=(224, 224, 3)))

headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
    layer.trainable = False

In [17]:
opt = Adam(lr=learning_rate, decay=learning_rate / epochs)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])

In [18]:
model.fit(
    aug.flow(trainX, trainY, batch_size=batchs),
    steps_per_epoch=len(trainX) // batchs,
    validation_data=(testX, testY),
    validation_steps=len(testX) // batchs,
    epochs=epochs)

Epoch 1/20
95/95 [==============================] - 152s 2s/step - loss: 0.3828 - accuracy: 0.8622 - val_loss: 0.1400 - val_accuracy: 0.9857
Epoch 2/20
95/95 [==============================] - 155s 2s/step - loss: 0.1502 - accuracy: 0.9624 - val_loss: 0.0756 - val_accuracy: 0.9883
Epoch 3/20
95/95 [==============================] - 151s 2s/step - loss: 0.1053 - accuracy: 0.9736 - val_loss: 0.0566 - val_accuracy: 0.9896
Epoch 4/20
95/95 [==============================] - 158s 2s/step - loss: 0.0824 - accuracy: 0.9786 - val_loss: 0.0484 - val_accuracy: 0.9909
Epoch 5/20
95/95 [==============================] - 154s 2s/step - loss: 0.0700 - accuracy: 0.9806 - val_loss: 0.0422 - val_accuracy: 0.9935
Epoch 6/20
95/95 [==============================] - 147s 2s/step - loss: 0.0572 - accuracy: 0.9868 - val_loss: 0.0417 - val_accuracy: 0.9896
Epoch 7/20
95/95 [==============================] - 151s 2s/step - loss: 0.0584 - accuracy: 0.9825 - val_loss: 0.0361 - val_accuracy: 0.9922
Epoch 8/20
95

In [19]:
predictions = model.predict(testX, batch_size=batchs)

predictions = np.argmax(predictions, axis=1)

24/24 [==============================] - 22s 827ms/step


In [20]:
cls_report = classification_report(testY.argmax(axis=1), predictions,
                            target_names=lb.classes_)
print(cls_report)

              precision    recall  f1-score   support

   with_mask       0.99      0.99      0.99       383
without_mask       0.99      0.99      0.99       384

    accuracy                           0.99       767
   macro avg       0.99      0.99      0.99       767
weighted avg       0.99      0.99      0.99       767



In [22]:
model.save("models/mask_detector_model.h5")